In [4]:
import pygame
import neat
import time
import os
import random 


In [5]:
Bird_IMG=[pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird1.png"))),pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird2.png"))),pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird3.png")))]
Pipe_IMG=pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","pipe.png")))
Base_IMG=pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","base.png")))
BG_IMG=pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bg.png")))

In [6]:

WIN_WIDTH = 550
WIN_HEIGHT = 750


class Bird:
    Img=Bird_IMG
    Max_Rotation=25
    Rot_vel=20
    Animation_Time=5

    def __init__(self,x,y):
        self.x=x
        self.y=y
        self.tilt=0
        self.tick_count=0
        self.vel=0
        self.height=self.y
        self.img_count=0
        self.img=self.Img[0]

    def jump(self):
        self.tick_count=0
        self.vel=-10.5
        self.height=self.y

    def move(self):
        self.tick_count += 1

        d=self.vel*self.tick_count + 1.5*self.tick_count**2

        if d>=16:
            d=16

        if d<0:
            d=-2

        self.y=self.y+d

        if d<0 or self.y<self.height+50:
            if self.tilt<self.Max_Rotation:
                self.tilt=self.Max_Rotation

        else:
            if self.tilt>-90:
                self.tilt -= self.Rot_vel

    def draw(self,win):
        self.img_count +=1

        if self.img_count<self.Animation_Time:
            self.img=self.Img[0]

        if self.img_count<self.Animation_Time*2:
            self.img=self.Img[1]

        if self.img_count<self.Animation_Time*3:
            self.img=self.Img[2]

        if self.img_count<self.Animation_Time*4:
            self.img=self.Img[1]
            
        if self.img_count==self.Animation_Time*4+1:
            self.img=self.Img[0]
            self.img_count=0

        

        blitRotateCenter(win, self.img, (self.x, self.y), self.tilt)

    def get_mask(self):
        return pygame.mask.from_surface(self.img)


def blitRotateCenter(surf, image, topleft, angle):
   
    rotated_image = pygame.transform.rotate(image, angle)
    new_rect = rotated_image.get_rect(center = image.get_rect(topleft = topleft).center)

    surf.blit(rotated_image, new_rect.topleft)
    
def draw_window(win,birds,Pipe,base):
    
    win.blit(BG_IMG,(0,0))

    for pipe in Pipe:
        pipe.draw(win)

    base.draw(win)

    for bird in birds:
        bird.draw(win)
        
    
    pygame.display.update()

class Pipe():
   
    GAP = 200
    VEL = 5

    def __init__(self, x):
      
        self.x = x
        self.height = 0

        # where the top and bottom of the pipe is
        self.top = 0
        self.bottom = 0

        self.PIPE_TOP = pygame.transform.flip(Pipe_IMG, False, True)
        self.PIPE_BOTTOM = Pipe_IMG

        self.passed = False

        self.set_height()

    def set_height(self):
        self.height = random.randrange(50, 450)
        self.top = self.height - self.PIPE_TOP.get_height()
        self.bottom = self.height + self.GAP

    def move(self):
      
        self.x -= self.VEL

    def draw(self, win):
      
        # draw top
        win.blit(self.PIPE_TOP, (self.x, self.top))
        # draw bottom
        win.blit(self.PIPE_BOTTOM, (self.x, self.bottom))

    def collide(self, bird):
        
        bird_mask = bird.get_mask()
        top_mask = pygame.mask.from_surface(self.PIPE_TOP)
        bottom_mask = pygame.mask.from_surface(self.PIPE_BOTTOM)
        top_offset = (self.x - bird.x, self.top - round(bird.y))
        bottom_offset = (self.x - bird.x, self.bottom - round(bird.y))

        b_point = bird_mask.overlap(bottom_mask, bottom_offset)
        t_point = bird_mask.overlap(top_mask,top_offset)

        if b_point or t_point:
            return True

        return False

class Base:
    
    VEL = 5
    WIDTH = Base_IMG.get_width()
    IMG = Base_IMG

    def __init__(self, y):
        
        self.y = y
        self.x1 = 0
        self.x2 = self.WIDTH
    def move(self):
        
        self.x1 -= self.VEL
        self.x2 -= self.VEL
        if self.x1 + self.WIDTH < 0:
            self.x1 = self.x2 + self.WIDTH

        if self.x2 + self.WIDTH < 0:
            self.x2 = self.x1 + self.WIDTH

    def draw(self, win):
        """
        Draw the floor. This is two images that move together.
        :param win: the pygame surface/window
        :return: None
        """
        win.blit(self.IMG, (self.x1, self.y))
        win.blit(self.IMG, (self.x2, self.y))
    
def main(genomes,config):
    nets=[]
    ge=[]
    birds=[]

    for _,g in genomes:
        net=neat.nn.FeedForwardNetwork.create(g,config)
        nets.append(net)
        birds.append(Bird(230,350))
        g.fitness=0
        ge.append(g)
        

    
    
    base=Base(730)
    pipes =[Pipe(700)]
    win = pygame.display.set_mode((WIN_WIDTH,WIN_HEIGHT))
    clock = pygame.time.Clock()
    
    run=True
    while run:
        clock.tick(30)
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                run=False
                pygame.quit()
                quit()
                
        pipe_ind=0
        if len(birds)>0:
            if len(pipes)>1 and birds[0].x>pipes[0].x + pipes[0].PIPE_TOP.get_width():
                pipe_ind=1
        else:
            # run=False
            # break
            

        for x,bird in enumerate(birds):
            bird.move()
            ge[x].fitness +=0.1

            output=nets[x].activate((bird.y,abs(bird.y-pipes[pipe_ind].height),abs(bird.y-pipes[pipe_ind].bottom)))

            if output[0]>0.5:
                bird.jump()

                
        add_pipe=False
        rem=[]
        for pipe in pipes:
            for x,bird in enumerate(birds):
                
                if pipe.collide(bird):
                    ge[x].fitness-=1
                    birds.pop(x)
                    nets.pop(x)
                    ge.pop(x)
           
                if not pipe.passed and pipe.x<bird.x:
                    pipe.passed=True
                    add_pipe=True

            if pipe.x + pipe.PIPE_TOP.get_width()<0:
                rem.append(pipe)
            pipe.move()
            
        if add_pipe:
            for g in ge:
                g.fitness +=5
            pipes.append(Pipe(700))

        for r in rem:
            pipes.remove(r)

        for x,bird in enumerate(birds):
            if bird.y + bird.img.get_height()>=730 or bird.y<0 :
                birds.pop(x)
                nets.pop(x)
                ge.pop(x)
                
        base.move()
        
        draw_window(win,birds,pipes,base)


    
    

def run(config_path):
      config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)

      p=neat.Population(config)
      p.add_reporter(neat.StdOutReporter(True))
      stats=neat.StatisticsReporter()
      p.add_reporter(stats)

      winner=p.run(main,50)
if __name__=="__main__":
    # local_dir=os.path.dirname(__file__)
    config_path=os.path.join("E:\Machine learning\Projects\ML(Tech_Tim)\Flapyy_Bird_AI","config-feedforward.txt")
    run(config_path)
        

IndentationError: expected an indented block (<ipython-input-6-531aea652fc4>, line 217)